# Dust_Mean_STD - with TABLE
- This outputs the dust mean and STD values given RA, DEC and a relavent field of view
- In a table

In [ ]:
"""IMPORTS"""
from astropy import units as u
from astroquery.irsa_dust import IrsaDust
import astropy.units as u
import numpy as np
import matplotlib.patches as patches
import matplotlib
from astropy.table import Table, Column

"""Parameters
ra=
dec=
field of view="""


def dust_mean_std(ra, dec, field_of_view=None):
    """Input args are Right Ascension (RA) and Declination(DEC) and must be in format [e.g: -29 51 56.74]"""
    if field_of_view == None:
        dust_map = IrsaDust.get_images("%s %s" %(ra, dec), image_type="ebv", radius=2.6 * u.deg)
    else:
        dust_map = IrsaDust.get_images("%s %s" %(ra, dec), image_type="ebv", radius=field_of_view * u.deg)
    dust=[]
    for f in dust_map:
        dust.append(np.array((f)[0].data, dtype=float))
    dust_std=round(np.std(dust),4)
    dust_mean=round(np.mean(dust), 4)
    data_rows = [(dust_mean, dust_std)]
    t = Table(rows=data_rows, names=('Dust Mean [E(B-V)]', 'Dust STD [E(B-V)]'))
    print ()
    print(t)
    print ()
    print ()


In [ ]:
"""IMPORTS"""
from astropy.coordinates import SkyCoord
import astropy.units as u

"""Parameters
ra='13 37 00'
dec='-29 51 56.74'"""


def lat_long(ra, dec):
    """Input args are Right Ascension (RA) and Declination(DEC) and must be in format [e.g: -29 51 56.74]"""
    lb=SkyCoord('%s %s'%(ra, dec), unit=(u.hourangle,u.deg))
    b=lb.galactic.b
    l=lb.galactic.l
    print ()
    print ("Galactic Latitude (b):  ", b)
    print ("Galactic Longitude (l): ", l)
    print ()
    print ()